In [766]:
import modern_robotics as mr
import numpy as np
import sympy as sp
from sympy import*
from sympy.physics.mechanics import dynamicsymbols, mechanics_printing
mechanics_printing()

In [767]:
def PsFromTsd(T_sd):
    #T_sd gir konfigurasjonen vi vil ha end-effector framen, B, i.
    #B, og derav også M, er lik som i DH
    #s er plassert nederst på roboten med positiv z oppover, altså ikke som i DH. Bør kanskje endres til å være lik DH 
    #T_sb = np.array([[0,0,-1,0],
    #                [0,1,0,0],
    #                 [1,0,0,0],
    #                 [0,0,0,1]])
    #T_sb = T_sd@T_db
    P_d = np.array([0,0,80,1])
    P_s = T_sd@P_d

    return P_s

def exp3(omega, theta):
    omega = skew(omega)
    R = sp.eye(3) + sp.sin(theta) * omega + (1 - sp.cos(theta)) * omega * omega
    return R

def skew(v):
    return Matrix([[0, -v[2], v[1]],
                    [v[2], 0, -v[0]],
                    [-v[1], v[0], 0]])
                    
def exp6(twist, theta):
    omega = skew(twist[:3])
    v = Matrix(twist[3:])
    T = eye(4)
    T[:3,:3] = exp3(twist[:3], theta)
    T[:3,3] = (eye(3) * theta + (1 - cos(theta)) * omega +
              (theta-sin(theta)) * omega * omega) * v
    return T


def rotX(alfa_im1):
    Rx = sp.eye(4)
    Rx[1,1] =    sp.cos(alfa_im1)
    Rx[1,2] =   -sp.sin(alfa_im1)
    Rx[2,1] =    sp.sin(alfa_im1)
    Rx[2,2] =    sp.cos(alfa_im1)
    return Rx

def rotZ(alfa_im1):
    Rz = sp.eye(4)
    Rz[0,0] =    sp.cos(alfa_im1)
    Rz[0,1] =   -sp.sin(alfa_im1)
    Rz[1,0] =    sp.sin(alfa_im1)
    Rz[1,1] =    sp.cos(alfa_im1)
    return Rz

def rotY(alfa_im1):
    Ry = sp.eye(4)
    Ry[0,0] =    sp.cos(alfa_im1)
    Ry[0,2] =   sp.sin(alfa_im1)
    Ry[2,0] =    -sp.sin(alfa_im1)
    Ry[2,2] =    sp.cos(alfa_im1)
    return Ry




In [1053]:
S1 = np.array([0,0,-1,0,0,0])
S2 = np.array([0,1,0,-400,0,25])
S3 = np.array([0,1,0,-855,0,25])
S4 = np.array([-1,0,0,0,-890,0])
S5 = np.array([0,1,0,-890,0,445])
S6 = np.array([-1,0,0,0,-890,0])
Slist = np.array([S1,S2,S3,S4,S5,S6]).T
print(Slist)

M = np.array([[0,0,-1,525],
              [0,1,0,0],
              [1,0,0,890],
              [0,0,0,1]])

thetas = [0,0,0,0,0,0]

[[   0    0    0   -1    0   -1]
 [   0    1    1    0    1    0]
 [  -1    0    0    0    0    0]
 [   0 -400 -855    0 -890    0]
 [   0    0    0 -890    0 -890]
 [   0   25   25    0  445    0]]


In [1232]:

#OPPGAVE 3.2

#Her endres T_sd for å teste andre gyldige konfigurasjoner:
#T_sd = M
thetas_gen = np.array([0,0,-np.pi/2,0,0,np.pi])
T_sd = mr.FKinSpace(M,Slist,thetas_gen)
print("T_sd\n", T_sd)

P_s = PsFromTsd(T_sd)
print("Ps", P_s)

#theta1
print("atan2 arg:",P_s[1], P_s[0])
thetas[0] = -atan2(P_s[1],P_s[0]) #minus foran fordi vinkelen er definert andre vei
print(N(thetas[0]), thetas_gen[0])

#theta2 and theta3
a = np.sqrt(420**2+35**2)
c = 455
P_merket = np.array([P_s[0],P_s[1],P_s[2]-400])
b = np.sqrt((np.sqrt(P_merket[0]**2+P_merket[1]**2)-25)**2 + P_merket[2]**2)
print("P_merket:", P_merket)

psi = np.arccos(420/a) #Se notatbok
phi = atan2(P_merket[2], sqrt(P_merket[0]**2 + P_merket[1]**2)-25)

alpha = np.arccos((b**2+c**2-a**2)/(2*b*c))
beta = np.arccos((a**2+c**2-b**2)/(2*a*c))

thetas[1] = np.pi/2 - (alpha + phi)

thetas[2] = np.pi/2 - (beta-psi)

print(thetas[1], thetas[2])

#Vi har XYX euler angles i stedet for ZYX som gitt i forelesning. De er (-X)Y(-X) fordi det er slik S'ene er definert, og det er viktig at de er definert konsekvent

T1 = exp6(S1, -thetas[0])
T2 = exp6(S2, -thetas[1])
T3 = exp6(S3, -thetas[2])


R = (T3@T2@T1@T_sd@np.linalg.inv(M)) #R er den resterende rotasjonen som trengs fra de tre siste leddene, definert i s

thetas[3] = -atan2(R[1,0], -R[2,0]) #minus foran theta4 og 6 fordi de er i minus x retning
thetas[4] = atan2(sqrt(1-R[0,0]**2), R[0,0])
thetas[5] = -atan2(R[0,1], R[0,2])

T_sd
 [[   1.    0.   -0.  -10.]
 [   0.   -1.    0.    0.]
 [  -0.   -0.   -1. 1355.]
 [   0.    0.    0.    1.]]
Ps [ -10.    0. 1275.    1.]
atan2 arg: 1.0899356512411443e-13 -9.999999999999895
-3.14159265358978 0.0
P_merket: [-10.   0. 875.]
3.07148097654389 - pi -1.3774936771703779


In [1233]:
#testing av analytisk løsning:
thetas_num = np.zeros(6)
thetas_deg = np.zeros(6)
for i in range(0,6):
    thetas_num[i] = N(thetas[i])
    thetas_deg[i] = np.rad2deg(thetas_num[i])

#print(thetas_deg, np.rad2deg(thetas_gen))
print(thetas_num, thetas_gen)

T_thetas = mr.FKinSpace(M,Slist,thetas_num)
P_reached = PsFromTsd(T_thetas)
#print(P_reached, P_s)
np.set_printoptions(precision=4)

fk_test = exp6(S4,thetas_num[3])@exp6(S5,thetas_num[4])@exp6(S6,thetas_num[5])@M
R_test = rotX(thetas_num[3])@rotY(thetas_num[4])@rotX(thetas_num[5]) 

thetas_calc = mr.IKinSpace(Slist,M,T_sd,[0,0,1,3,0,0],0.01,0.01)
#for i in i range(0,6)
  #  thetas_calc
print(thetas_calc)

T_sd, T_thetas

[-3.1416 -0.0701 -1.3775  3.1416  0.1232  3.1416] [ 0.      0.     -1.5708  0.      0.      3.1416]
(array([-0.0005,  0.    , -1.5708,  1.067 ,  0.    ,  2.0751]), True)


(array([[   1.,    0.,   -0.,  -10.],
        [   0.,   -1.,    0.,    0.],
        [  -0.,   -0.,   -1., 1355.],
        [   0.,    0.,    0.,    1.]]),
 array([[   1.,    0.,   -0.,  -10.],
        [   0.,   -1.,   -0.,    0.],
        [  -0.,    0.,   -1., 1355.],
        [   0.,    0.,    0.,    1.]]))

In [1234]:
sy = np.array([0,1,0,0,0,0])
sz = np.array([0,0,1,0,0,0])
T1 = exp6(S1, thetas[0])
T2 = exp6(S2, thetas[1])
T3 = exp6(S3, thetas[2])

print(T_sd)


fk_test = exp6(S4,np.pi/2)@exp6(S5, np.pi/2)
R_test = rotZ(np.pi/2)@rotY(np.pi/2)
print(M)
print(T_sd)

R_test, fk_test


[[   1.    0.   -0.  -10.]
 [   0.   -1.    0.    0.]
 [  -0.   -0.   -1. 1355.]
 [   0.    0.    0.    1.]]
[[  0   0  -1 525]
 [  0   1   0   0]
 [  1   0   0 890]
 [  0   0   0   1]]
[[   1.    0.   -0.  -10.]
 [   0.   -1.    0.    0.]
 [  -0.   -0.   -1. 1355.]
 [   0.    0.    0.    1.]]


⎛⎡3.74939945665464e-33          -1.0          6.12323399573677e-17  0⎤  ⎡1.110
⎜⎢                                                                   ⎥  ⎢     
⎜⎢6.12323399573677e-17  6.12323399573677e-17          1.0           0⎥  ⎢     
⎜⎢                                                                   ⎥, ⎢     
⎜⎢        -1.0                   0            6.12323399573677e-17  0⎥  ⎢-1.11
⎜⎢                                                                   ⎥  ⎢     
⎝⎣         0                     0                     0            1⎦  ⎣     

22302462516e-16            0                     1            -445.0⎤⎞
                                                                    ⎥⎟
    -1            1.11022302462516e-16  1.11022302462516e-16  445.0 ⎥⎟
                                                                    ⎥⎟
022302462516e-16           -1           1.23259516440783e-32  890.0 ⎥⎟
                                                                    ⎥⎟
     0              